In [1]:
# sentiment_model.py

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model + tokenizer once at the top
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Class labels from SST-2
label_map = {0: "negative", 1: "positive"}

def predict_sentiment(text):
    """
    Run sentiment analysis on input text using a pretrained DistilBERT model.

    Returns:
        - label: "positive" or "negative"
        - confidence: softmax probability of prediction
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0][pred_class].item()

    return label_map[pred_class], round(confidence, 3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# ---- data_stream.py logic ----
import random
import time

TWEET_BANK = [
    "I love how this phone feels in my hand!",
    "This update completely broke my app...",
    "I’m not sure how I feel about this feature.",
    "What an amazing performance! I’m blown away.",
    "The food was cold and overpriced. Never going back.",
    "Just had the best customer service experience!",
    "Ugh. Delayed again. Thanks, airline.",
    "Honestly, that trailer gave me chills.",
    "Worst. Day. Ever.",
    "Feeling grateful for the little things today."
]

def stream_tweets(interval=2):
    while True:
        yield random.choice(TWEET_BANK)
        time.sleep(interval)


In [14]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 84.1 MB/s eta 0:00:00


In [15]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import random

# Load sentiment model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

label_map = {0: "negative", 1: "positive"}

# Simulated tweets
tweet_bank = [
    "I love how this phone feels in my hand!",
    "This update completely broke my app...",
    "What an amazing performance!",
    "Worst. Day. Ever.",
    "Ugh. Delayed again. Thanks, airline.",
    "Feeling grateful for the little things today.",
    "I can't believe how good this is.",
    "Absolutely trash. I'm so disappointed.",
    "Just made my day 10x better.",
    "Customer service has been horrible lately."
]

def get_new_sentiments():
    history = []
    for _ in range(10):
        tweet = random.choice(tweet_bank)
        inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            pred_class = torch.argmax(probs, dim=1).item()
            confidence = round(probs[0][pred_class].item(), 3)
        sentiment = label_map[pred_class]
        history.append([tweet, sentiment, f"{confidence:.2f}"])
    return history[::-1]

# Gradio UI — works on Hugging Face Spaces
with gr.Blocks() as demo:
    gr.Markdown("# 📊 Real-Time Sentiment Dashboard")
    gr.Markdown("Click the button below to simulate a new batch of tweet sentiment predictions.")
    btn = gr.Button("🔁 Refresh")
    output = gr.Dataframe(headers=["Tweet", "Sentiment", "Confidence"])
    btn.click(fn=get_new_sentiments, inputs=[], outputs=output)

demo.launch()

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://77de4e31aeae15e57c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
!ngrok config add-authtoken 2wuu7g07NH6qt1Shhm54khRPnZP_7kHxSskqNqYQmuX6T3vEx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

# Apply the asyncio patch for Colab
nest_asyncio.apply()

# Set up a tunnel to the FastAPI app
public_url = ngrok.connect(8000)
print("Your live API is here:", public_url)

# Launch the app
uvicorn.run(app, host="0.0.0.0", port=8000)

Your live API is here: NgrokTunnel: "https://bde8-34-91-9-190.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [5272]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2603:6010:2100:37e3:8d2b:1f3e:1934:1ebe:0 - "GET / HTTP/1.1" 200 OK
INFO:     2603:6010:2100:37e3:8d2b:1f3e:1934:1ebe:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5272]


In [11]:
import os
from huggingface_hub import HfApi, Repository, whoami, login

# Step 1: Login (you'll be prompted to paste your Hugging Face token)
login()

In [16]:
# Step 2: Config
SPACE_NAME = "realtime-sentiment-dashboard"
api = HfApi()
username = whoami()["name"]
full_repo_name = f"{username}/{SPACE_NAME}"

# Step 3: Create the Space if it doesn't exist
repo_url = api.create_repo(
    repo_id=full_repo_name,
    repo_type="space",
    space_sdk="gradio",
    exist_ok=True
)

# Step 4: Clone the empty repo into a fresh directory
LOCAL_DIR = "./sentiment_dashboard"
if os.path.exists(LOCAL_DIR):
    import shutil
    shutil.rmtree(LOCAL_DIR)

repo = Repository(
    local_dir=LOCAL_DIR,
    clone_from=repo_url
)

# Step 5: Write app.py
with open(os.path.join(LOCAL_DIR, "app.py"), "w") as f:
    f.write("""\
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import random

# Load sentiment model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

label_map = {0: "negative", 1: "positive"}
tweet_bank = [
    "I love how this phone feels in my hand!",
    "This update completely broke my app...",
    "What an amazing performance!",
    "Worst. Day. Ever.",
    "Ugh. Delayed again. Thanks, airline.",
    "Feeling grateful for the little things today.",
    "Absolutely trash. I'm so disappointed.",
    "Just made my day 10x better.",
    "Customer service has been horrible lately."
]

def get_new_sentiments():
    history = []
    for _ in range(10):
        tweet = random.choice(tweet_bank)
        inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            pred_class = torch.argmax(probs, dim=1).item()
            confidence = round(probs[0][pred_class].item(), 3)
        sentiment = label_map[pred_class]
        history.append([tweet, sentiment, f"{confidence:.2f}"])
    return history[::-1]

with gr.Blocks() as demo:
    gr.Markdown("# 📊 Real-Time Sentiment Dashboard")
    gr.Markdown("Click the button below to simulate a new batch of tweet sentiment predictions.")
    btn = gr.Button("🔁 Refresh")
    output = gr.Dataframe(headers=["Tweet", "Sentiment", "Confidence"])
    btn.click(fn=get_new_sentiments, inputs=[], outputs=output)

demo.launch()
""")

# Step 6: Write requirements.txt
with open(os.path.join(LOCAL_DIR, "requirements.txt"), "w") as f:
    f.write("transformers\ntorch\ngradio\n")

# Step 7: Push to Hugging Face
repo.push_to_hub(commit_message="🚀 Deploy real-time sentiment dashboard")
print(f"🎉 Deployed! Your Space is live at: https://huggingface.co/spaces/{full_repo_name}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/spaces/zda23/realtime-sentiment-dashboard into local empty directory.
To https://huggingface.co/spaces/zda23/realtime-sentiment-dashboard
   22bf19b..a4a0d49  main -> main

   22bf19b..a4a0d49  main -> main



🎉 Deployed! Your Space is live at: https://huggingface.co/spaces/zda23/realtime-sentiment-dashboard
